In [1]:
import json
from copy import deepcopy
import numpy as np
import pandas as pd
import shutil
import pickle

In [2]:
def gen_ann_file(target, shot):

    dict_ann = {}

    for split in ["train", "val"]:
            
        df_label = pd.read_csv(f"./MedFMC/{target}/{target}_{split}.csv")
        if target == "chest":
            df_label.drop(columns=['Unnamed: 0'], inplace=True)
        elif target == "colon":
            df_label.drop(columns=['slide_id'], inplace=True)
        elif target == "endo":
            df_label.drop(columns=['Unnamed: 0', 'study_id'], inplace=True)
        df_label.set_index('img_id', inplace=True)

        
        
        # display(df_label)
        
        with open(f"data_backup/{target}_{shot}-shot_{split}_exp1.txt") as file:
            dict_ann[split] = file.readlines()
            # print(dict_ann[split])
    
        for i_ele, ele in enumerate(dict_ann[split]):
            dict_ann[split][i_ele] = [i.strip('\n') for i in ele.split(' ')]
        # print(dict_ann[split])
    
        # for ele in dict_ann[split]:
        #     print((df_label[df_label.index == ele[0]].values.flatten() == np.array(ele[1].split(',')).astype(float)).all(), end=', ')

    with open(f"data_backup/{target}_test_WithLabel.txt") as file:
        dict_ann['test'] = file.readlines()

    # print(dict_ann)
    
    for i_ele, ele in enumerate(dict_ann['test']):
        dict_ann['test'][i_ele] = [i.strip('\n') for i in ele.split(' ')] # [['a.png', '0,1,0,1'], ['b.png', '1,1,0,0']]
    
    
    # --------------------------- CoCoOp ---------------------------
    
    for i_col, col in enumerate(df_label.columns):
        # print(i_col, col)

        dict_ann_col = deepcopy(dict_ann)

        for split in ["train", "val", "test"]:
            for i_ele, ele in enumerate(dict_ann_col[split]):

                if target == "chest":
                    dict_ann_col[split][i_ele][1] = int(ele[1].split(',')[i_col]) # ['a.png', '0,1,0,1'] -> ['a.png', 0] / ['a.png', 1] / ['a.png', 0] / ['a.png', 0]
                elif target == "colon":
                    dict_ann_col[split][i_ele][1] = dict_ann_col[split][i_ele][0] + ' ' + dict_ann_col[split][i_ele][1]
                    dict_ann_col[split][i_ele] = dict_ann_col[split][i_ele][1:]
                    ele = dict_ann_col[split][i_ele]
                    dict_ann_col[split][i_ele][1] = int(ele[1].split(',')[i_col])
                elif target == "endo":
                    dict_ann_col[split][i_ele] = [dict_ann_col[split][i_ele][0], int(dict_ann_col[split][i_ele][i_col+1])] # ['a.png', '0', '1', '0', '1'] -> ['a.png', 0] / ['a.png', 1] / ['a.png', 0] / ['a.png', 0]
                    ele = dict_ann_col[split][i_ele]
                
                

                if dict_ann_col[split][i_ele][1] == 0:
                    dict_ann_col[split][i_ele].append(f"{col}-negative")
                elif dict_ann_col[split][i_ele][1] == 1:
                    dict_ann_col[split][i_ele].append(f"{col}-positive")
        
        
        with open(f"./MedFMC/{target}/split_tsai_medfmc_{target}-{shot}shot-{col}.json", "w") as file:
            json.dump(dict_ann_col, file)
    print(f"dict_ann_col.keys() = {dict_ann_col.keys()}")
    print(f"dict_ann_col['train'][0] = {dict_ann_col['train'][0]}")
    
    
    # --------------------------- VPT ---------------------------
    
    if target == "colon":
        for split in ["train", "val"]:
            shutil.copyfile(f"data_backup/{target}_{shot}-shot_{split}_exp1.txt", f"MedFMC/{target}/{target}_{shot}-shot_{split}_exp1.txt")
        shutil.copyfile(f"data_backup/{target}_train_20.txt", f"MedFMC/{target}/{target}_train_20.txt")
        shutil.copyfile(f"data_backup/{target}_val_20.txt", f"MedFMC/{target}/{target}_val_20.txt")
        shutil.copyfile(f"data_backup/{target}_test_WithLabel.txt", f"MedFMC/{target}/{target}_test_WithLabel.txt")
    elif target == "chest":
        
        for split in ["train", "val"]:
            dict_ann_vpt = {'metainfo': {'classes': list(df_label.columns)}, 'data_list': []}
            
            # data_ann['split']: [['a.png', '0,1,0,1'], ['b.png', '1,1,0,0']]
            
            for i_ele, ele in enumerate(dict_ann[split]):
                dict_ann_vpt['data_list'].append({'img_path': ele[0], 'gt_label': []})
                
                for j, jj in enumerate(ele[1].split(',')):
                    if jj == '1':
                        dict_ann_vpt['data_list'][-1]['gt_label'].append(j)
            print('<==========')
            print(f"dict_ann[{split}][0:2] = {dict_ann[split][0:2]}")
            print(f"dict_ann_vpt['metainfo'] = {dict_ann_vpt['metainfo']}")
            print(f"dict_ann_vpt['data_list'][0:2] = {dict_ann_vpt['data_list'][0:2]}")
            print('==========>')
            
            with open(f"MedFMC/{target}/{target}_{shot}-shot_{split}_exp1.pkl", 'wb') as file:
                pickle.dump(dict_ann_vpt, file)
        
            
    elif target == "endo":
        for split in ["train", "val"]:
            dict_ann_vpt = {'metainfo': {'classes': list(df_label.columns)}, 'data_list': []}
            
            # data_ann['split']: [['a.png', '0,1,0,1'], ['b.png', '1,1,0,0']]
            
            for i_ele, ele in enumerate(dict_ann[split]):
                dict_ann_vpt['data_list'].append({'img_path': ele[0], 'gt_label': []})
                
                for j, jj in enumerate(ele[1:]):
                    if jj == '1':
                        dict_ann_vpt['data_list'][-1]['gt_label'].append(j)
            print('<==========')
            print(f"dict_ann[{split}][0:2] = {dict_ann[split][0:2]}")
            print(f"dict_ann_vpt['metainfo'] = {dict_ann_vpt['metainfo']}")
            print(f"dict_ann_vpt['data_list'][0:2] = {dict_ann_vpt['data_list'][0:2]}")
            print('==========>')
            
            with open(f"MedFMC/{target}/{target}_{shot}-shot_{split}_exp1.pkl", 'wb') as file:
                pickle.dump(dict_ann_vpt, file)
    
    print('\n\n---------------------------------------------------------------------\n\n')

In [3]:
for target in ["chest", "colon", "endo"]:
    for shot in [1, 5, 10]:
        print(target, shot)
        gen_ann_file(target, shot)

chest 1
dict_ann_col.keys() = dict_keys(['train', 'val', 'test'])
dict_ann_col['train'][0] = ['DX.1.2.392.200046.100.2.1.69131194618.191116084230.1.1.1.png', 0, 'consolidation-negative']
<==========
dict_ann[train][0:2] = [['DX.1.2.392.200046.100.2.1.69131194618.191116084230.1.1.1.png', '1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0'], ['5C2EB54413D4CEC.png', '0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0']]
dict_ann_vpt['metainfo'] = {'classes': ['pleural_effusion', 'nodule', 'pneumonia', 'cardiomegaly', 'hilar_enlargement', 'fracture_old', 'fibrosis', 'aortic_calcification', 'tortuous_aorta', 'thickened_pleura', 'TB', 'pneumothorax', 'emphysema', 'atelectasis', 'calcification', 'pulmonary_edema', 'increased_lung_markings', 'elevated_diaphragm', 'consolidation']}
dict_ann_vpt['data_list'][0:2] = [{'img_path': 'DX.1.2.392.200046.100.2.1.69131194618.191116084230.1.1.1.png', 'gt_label': [0, 4]}, {'img_path': '5C2EB54413D4CEC.png', 'gt_label': [1, 5, 7]}]
==========>
<==========
dict_ann[val][0:2] = [['5

In [4]:
target = "chest"

for split in ["train", "val", 'test']:
    
    if split == "train":
        file_name = f"{target}_train_20.txt"
    elif split == "val":
        file_name = f"{target}_val_20.txt"
    elif split == "test":
        file_name = f"{target}_test_WithLabel.txt"
        
    print(split)

    dict_ann_vpt = {'metainfo': {'classes': ['pleural_effusion', 'nodule', 'pneumonia', 'cardiomegaly', 'hilar_enlargement', 'fracture_old', 'fibrosis', 'aortic_calcification', 'tortuous_aorta', 'thickened_pleura', 'TB', 'pneumothorax', 'emphysema', 'atelectasis', 'calcification', 'pulmonary_edema', 'increased_lung_markings', 'elevated_diaphragm', 'consolidation']}, 'data_list': []}

    with open(f"data_backup/{file_name}") as file:
        data = file.readlines()

    for i_ele, ele in enumerate(data):
        data[i_ele] = [i.strip('\n') for i in ele.split(' ')]

    for i_ele, ele in enumerate(data):
        dict_ann_vpt['data_list'].append({'img_path': ele[0], 'gt_label': []})

        for j, jj in enumerate(ele[1].split(',')):
            if jj == '1':
                dict_ann_vpt['data_list'][-1]['gt_label'].append(j)
    print('<==========')
    print(f"data[0:2] = {data[0:2]}")
    # print(f"dict_ann_vpt['metainfo'] = {dict_ann_vpt['metainfo']}")
    print(f"dict_ann_vpt['data_list'][0:2] = {dict_ann_vpt['data_list'][0:2]}")
    print('==========>')

    with open(f"MedFMC/{target}/{file_name.replace('txt', 'pkl')}", 'wb') as file:
        pickle.dump(dict_ann_vpt, file)

train
<==========
data[0:2] = [['5D0984CC1D7C36B.png', '1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0'], ['5E23EE5AF64A05.png', '0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0']]
dict_ann_vpt['data_list'][0:2] = [{'img_path': '5D0984CC1D7C36B.png', 'gt_label': [0, 2, 4, 11]}, {'img_path': '5E23EE5AF64A05.png', 'gt_label': [1, 14]}]
==========>
val
<==========
data[0:2] = [['CR.1.3.12.2.1107.5.3.57.20200.11.201910300851500343.png', '0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0'], ['5CF728A22460CBA.png', '1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0']]
dict_ann_vpt['data_list'][0:2] = [{'img_path': 'CR.1.3.12.2.1107.5.3.57.20200.11.201910300851500343.png', 'gt_label': [3]}, {'img_path': '5CF728A22460CBA.png', 'gt_label': [0, 2, 11]}]
==========>
test
<==========
data[0:2] = [['5B91F7409CCCE2.png', '0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0'], ['5DC2824164E8B52.png', '1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0']]
dict_ann_vpt['data_list'][0:2] = [{'img_path': '5B91F7409CCCE2.png', 'gt_label': [16]}, {'img_path': '5DC2824164E8B5

In [5]:
target = "endo"

for split in ["train", "val", 'test']:
    
    if split == "train":
        file_name = f"{target}_train_20.txt"
    elif split == "val":
        file_name = f"{target}_val_20.txt"
    elif split == "test":
        file_name = f"{target}_test_WithLabel.txt"
        
    print(split)

    dict_ann_vpt = {'metainfo': {'classes': ['ulcer', 'erosion', 'polyp', 'tumor']}, 'data_list': []}

    with open(f"data_backup/{file_name}") as file:
        data = file.readlines()

    for i_ele, ele in enumerate(data):
        data[i_ele] = [i.strip('\n') for i in ele.split(' ')]

    for i_ele, ele in enumerate(data):
        dict_ann_vpt['data_list'].append({'img_path': ele[0], 'gt_label': []})

        for j, jj in enumerate(ele[1:]):
            if jj == '1':
                dict_ann_vpt['data_list'][-1]['gt_label'].append(j)
    print('<==========')
    print(f"data[0:2] = {data[0:2]}")
    # print(f"dict_ann_vpt['metainfo'] = {dict_ann_vpt['metainfo']}")
    print(f"dict_ann_vpt['data_list'][0:2] = {dict_ann_vpt['data_list'][0:2]}")
    print('==========>')

    with open(f"MedFMC/{target}/{file_name.replace('txt', 'pkl')}", 'wb') as file:
        pickle.dump(dict_ann_vpt, file)

train
<==========
data[0:2] = [['13333_2021.09_0003_51743305.png', '0', '0', '0', '0'], ['13333_2021.09_0004_52720383.png', '0', '0', '0', '0']]
dict_ann_vpt['data_list'][0:2] = [{'img_path': '13333_2021.09_0003_51743305.png', 'gt_label': []}, {'img_path': '13333_2021.09_0004_52720383.png', 'gt_label': []}]
==========>
val
<==========
data[0:2] = [['13333_2021.09_0004_52719617.png', '0', '0', '1', '0'], ['13333_2021.09_0004_52720500.png', '0', '0', '1', '0']]
dict_ann_vpt['data_list'][0:2] = [{'img_path': '13333_2021.09_0004_52719617.png', 'gt_label': [2]}, {'img_path': '13333_2021.09_0004_52720500.png', 'gt_label': [2]}]
==========>
test
<==========
data[0:2] = [['13333_2021.11_0003_55200268.png', '0', '0', '0', '0'], ['13333_2021.11_0003_55199880.png', '0', '0', '0', '0']]
dict_ann_vpt['data_list'][0:2] = [{'img_path': '13333_2021.11_0003_55200268.png', 'gt_label': []}, {'img_path': '13333_2021.11_0003_55199880.png', 'gt_label': []}]
==========>
